# Entrega final

**(Se requiere el notebook "Análisis de sueldos" resuelto)**

En base a los resultados obtenidos en *"Análisis de sueldos"* confeccionar conclusiones respecto a:

- Rol vs sueldos
- Nivel de educación alcanzada vs sueldos
- Género vs sueldos
- Tecnologías populares

Generar gráficos de barra para mostrar los resultados mencionados anteriormente usando strings, ej:

```
developer | -----------------------------------------------
sysadmin  | ---------------------------
QA        | ------------------
          | -----------------5%--------10%----------------40%
```

In [241]:
# FRAGMENTOS RESOLUCION DE "ANALISIS DE SUELDOS" NECESARIOS PARA RESPONDER A LAS PREGUNTAS
#Abrimos los archivos

file_name_1 = "2021.2 - sysarmy - Encuesta de remuneración salarial Argentina - Argentina.csv"
file_name_2 = "2021.2 - sysarmy - Encuesta de remuneración salarial Latam - Latam.csv"

#limpio primeras filas de los documentos
data_arg_temp = []
data_latam_temp = []

import csv
with open(file_name_1, 'r') as file_1:
    reader = csv.reader(file_1)
    for row in reader:
        data_arg_temp.append(row)
        
with open(file_name_2, 'r') as file_2:
    reader = csv.reader(file_2)
    for row in reader:
        data_latam_temp.append(row)
 
del data_arg_temp[:9]
del data_latam_temp[:11]

#Reacomoda las columnas referidas a ciudad-pais en latam
for case in data_latam_temp:
    i = 18
    while i <= 18 and i > 0:
        if len(case[i]) == 0:
            case.pop(i)
        i -= 1

#Agrego campo Pais en Argentina
for i in range(0,len(data_arg_temp)):
    if i==0:
        data_arg_temp[i].insert(0,"En que pais trabajas")
    else:
        data_arg_temp[i].insert(0,"Argentina")

#Uno ambas tablas en una
data_total_temp = data_arg_temp + data_latam_temp

file_1.close()
file_2.close()

#Guardo los csv unificado
with open('Data.csv', 'w') as file: 
    write = csv.writer(file) 
    write.writerow(data_total_temp[0])
    write.writerows(data_total_temp[1:])  

#Armo diccionario de respuestas
data = []
with open('Data.csv') as file_3:
    reader = csv.DictReader(file_3)
    for row in reader:
        data.append(row)
        
file_3.close()
#print("Elegi armar una lista de diccionarios para poder acceder facilmente a las respuestas segun su key")

#REGION----------------------------------------------------------------------------------------------------
#Porcentaje de participacion por region
#Determino funcion para generar set con valores unicos a cada pregunta de la encuesta
def respuestas_unicas(pregunta_encuesta):
    respuestas = set()
    for i in range(0,len(data)):
        if data[i][pregunta_encuesta] != '':
            respuestas.add(data[i][pregunta_encuesta])
    
    return respuestas

#Determino funcion para agregar cantidad de veces que se obtuvo cada respuesta
#y armo el diccionario de ciudades y participantes
def dic_respuestas(criterio, pregunta_encuesta):
    participantes = []
    for i in range(0,len(criterio)):
        participantes.append(0)
    
    criterio = list(criterio)
    dic_respuestas = dict(zip(criterio, participantes))
    
    for i in range(0,len(data)):
        for elemento in criterio:
            if data[i][pregunta_encuesta] == elemento:
                dic_respuestas[elemento] += 1

    return dic_respuestas
            

#Armo funcion para calcular participacion ordenadas de mayor a menor
def participacion_ordenada(dic_criterio, criterio): 

    #Calculo cantidad de respuestas como base
    cantidad_de_participantes = 0
    for elemento in criterio:
        cantidad_de_participantes += dic_criterio[elemento]
        
    #if cantidad_de_participantes == len(data):
        #print("CHEQUEO OK: CANTIDAD DE PARTICIPANTES EN LA PREGUNTA IGUAL AL TOTAL DE ENCUESTADOS")    
    
    #Calculo participacion
    dic_participacion = {}
    for elemento in criterio:
        dic_participacion[elemento]=round(dic_criterio[elemento]/cantidad_de_participantes*100,2)
   
    #Ordeno resultados de mayor a menor
    sorted_participacion = sorted(dic_participacion.values(), reverse =True)
    sorted_dic_participacion = {}

    for participacion in sorted_participacion:
        for elemento in dic_participacion.keys():
            if dic_participacion[elemento] == participacion:
                sorted_dic_participacion[elemento] = round(dic_participacion[elemento],2)
    
    return sorted_dic_participacion

#ROLES PARTICIPACION---------------------------------------------------------------------------------------------------------
#Porcentaje de participación por tipo de rol
#Uso las funciones definidas
roles = respuestas_unicas(pregunta_encuesta = "Trabajo de")
dic_roles = dic_respuestas(criterio = roles, pregunta_encuesta = "Trabajo de")
participacion_por_rol = participacion_ordenada(dic_criterio = dic_roles, criterio = roles)

#NIVEL EDUCATIVO PARTICIPACION---------------------------------------------------------------------------------------------------
#Armo diccionario de nivel educativo y participantes usando funciones
nivel_educativo = respuestas_unicas(pregunta_encuesta = "Nivel de estudios alcanzado")
dic_nivel_educativo = dic_respuestas(criterio = nivel_educativo, pregunta_encuesta = "Nivel de estudios alcanzado")
participacion_por_nivel = participacion_ordenada(dic_criterio = dic_nivel_educativo, criterio = nivel_educativo)
    
#print(f'Respuesta 7a - {participacion_por_nivel}')

#Armo diciconario de dicionarios con nivel y estado educativo

dic_estados = {'Completado': 0, 'Incompleto': 0, 'En curso': 0}
posiciones_estados = [['Completado', 0], ['Incompleto', 1], ['En curso', 2]]
estados_aux = ['Completado', 'Incompleto', 'En curso']

cantidades_por_estado = []    
for nivel in nivel_educativo:
    participantes_por_estado = [0, 0, 0]
    for i in range(0,len(data)):
        for estado in posiciones_estados:
            if data[i]["Nivel de estudios alcanzado"] == nivel and data[i]["Estado"] == estado[0]:
                participantes_por_estado[estado[1]] += 1
    for r in range (0,3):
        participantes_por_estado[r] = f'{round(participantes_por_estado[r]/dic_nivel_educativo[nivel]*100,2)}%'
    dic_participantes_por_estado = dict(zip(estados_aux, participantes_por_estado))
    dic_estado_educacion = {nivel: dic_participantes_por_estado}
    cantidades_por_estado.append(dic_estado_educacion)
    
#print(f'Respuesta 7b - {cantidades_por_estado}')


#GENEROS PARTICIPACION--------------------------------------------------------------------------------------------------------------
#Corregimos diferencias en respuestas
#Consideramos varon como varon cis y mujer como mujer cis
for i in range(0, len(data)):
    data[i]["Me identifico"] = data[i]["Me identifico"].capitalize()    
    data[i]["Me identifico"] = data[i]["Me identifico"].replace('á','a')
    data[i]["Me identifico"] = data[i]["Me identifico"].replace('ó','o')
    data[i]["Me identifico"] = data[i]["Me identifico"].replace('Varon Cis','Varon')
    data[i]["Me identifico"] = data[i]["Me identifico"].replace('Varon cis','Varon')
    data[i]["Me identifico"] = data[i]["Me identifico"].replace('Varon heterosexual','Varon')    
    data[i]["Me identifico"] = data[i]["Me identifico"].replace('Hombre','Varon')
    data[i]["Me identifico"] = data[i]["Me identifico"].replace('Masculino','Varon')
    data[i]["Me identifico"] = data[i]["Me identifico"].replace('Mujer Cis','Mujer')
    data[i]["Me identifico"] = data[i]["Me identifico"].replace('Mujer cis','Mujer')
    data[i]["Me identifico"] = data[i]["Me identifico"].replace('Varon trans','Varon-trans')
    data[i]["Me identifico"] = data[i]["Me identifico"].replace('Varon ','Varon')
    

#Traigo todas las respuestas a la pregunta
generos = respuestas_unicas(pregunta_encuesta = 'Me identifico')
    
#Depuramos respuestas que no aportan
generos_depurado = set()
for respuesta in generos:
    cantidad = 0
    for i in range(0,len(data)):
        if data[i]['Me identifico'] == respuesta:
            cantidad += 1
    if cantidad >2 and respuesta!= "Macho alfa" and respuesta !="Helicoptero de combate":
            generos_depurado.add(respuesta)
            
participantes_por_genero = []
for i in range(0,len(generos_depurado)):
    participantes_por_genero.append(0)

generos_depurado = list(generos_depurado)    
dic_generos = dict(zip(generos_depurado, participantes_por_genero))

#Cuento cantidad de participantes
for i in range(0,len(data)):
    for genero in generos_depurado:
        if data[i]["Me identifico"] == genero:
            dic_generos[genero] += 1

cant_respuestas_genero = sum(dic_generos.values())

#Armo diciconario con porcentajes de participacion 
dic_participacion_genero = dic_generos

#Calculo participacion
for genero in generos_depurado:
     dic_participacion_genero[genero]= round(dic_generos[genero]/cant_respuestas_genero*100,2)
#print(f'Respuesta 10a - Cantidad de participantes por genero: {dic_participacion_genero}')

#SUELDOS----------------------------------------------------------------------------------------------------
#Definir el tipo de cambio por pais para dolarizar todos los sueldos y asi poder compararlos
paises = set()
for i in range(0,len(data)):
    if len(data[i]["En que pais trabajas"]) != 0:
        paises.add(data[i]["En que pais trabajas"])

#Tomo TC del 27/01/2022 - en el caso de Argentina considero el oficial         
dic_TC ={'Argentina': 104.76,
 'Bolivia': 6.89,
 'Chile':805.25,
 'Colombia': 3963.01,
 'Costa Rica': 640.48,
 'Cuba': 24.02,
 'Ecuador': 1,
 'El Salvador': 8.75,
 'Guatemala': 7.70,
 'Honduras': 24.65,
 'México': 20.73,
 'Nicaragua': 35.49,
 'Panamá': 1,
 'Paraguay': 7055.63,
 'Perú': 3.85,
 'Puerto Rico': 1,
 'República Dominicana': 50.7,
 'Uruguay': 44.57,
 'Venezuela':4.58}        

#Me quedo solo con los valores en la columna sueldo bruto
#Primero identifico casos que usan punto decimal y lo cambio por coma para unificar formato
import re
for i in range(0, len(data)):
    if len(data[i]["Salario mensual o retiro BRUTO (en tu moneda local)"])>3:
        if data[i]["Salario mensual o retiro BRUTO (en tu moneda local)"][-2] == ".": 
            data[i]["Salario mensual o retiro BRUTO (en tu moneda local)"][-2].replace(".",",")   
        if  data[i]["Salario mensual o retiro BRUTO (en tu moneda local)"][-3] == '.':
            data[i]["Salario mensual o retiro BRUTO (en tu moneda local)"][-3].replace(".",",") 
                       
            
#Elimino todos los caracteres que no sean numeros o comas
for i in range(0, len(data)):
    data[i]["Salario mensual o retiro BRUTO (en tu moneda local)"] = re.sub("[^0-9,]", "", data[i]["Salario mensual o retiro BRUTO (en tu moneda local)"])

#Para poder operar convierto comas a puntos
for i in range(0, len(data)):
    data[i]["Salario mensual o retiro BRUTO (en tu moneda local)"] = data[i]["Salario mensual o retiro BRUTO (en tu moneda local)"].replace(",",".")   

for i in range(0, len(data)):
        if data[i]["Salario mensual o retiro BRUTO (en tu moneda local)"] == "." or data[i]["Salario mensual o retiro BRUTO (en tu moneda local)"] == "..": 
            data[i]["Salario mensual o retiro BRUTO (en tu moneda local)"].replace(".","") 

#creo columna con salario en usd            
for i in range(0, len(data)):
    for pais in paises:
        if data[i]['En que pais trabajas'] == pais:
            if data[i]['Salario mensual o retiro BRUTO (en tu moneda local)'] == ".":
                data[i]['Sueldo en dolares'] = ""
            elif data[i]['Salario mensual o retiro BRUTO (en tu moneda local)'] == "":
                 data[i]['Sueldo en dolares'] = ""    
            elif data[i]['Salario mensual o retiro BRUTO (en tu moneda local)'] != "":
                 data[i]['Sueldo en dolares'] = round((float(data[i]['Salario mensual o retiro BRUTO (en tu moneda local)'])/dic_TC[pais]),2)

def calculadora_mediana_sueldo(criterio, pregunta_encuesta):
    sueldo_criterio = []
    for i in range(0,len(criterio)):
        sueldo_criterio.append(0)

    criterio = list(criterio)
    dic_sueldo_criterio = dict(zip(criterio, sueldo_criterio))
    #print(dic_sueldo_criterio)
    for elemento in criterio:
        sueldos = []
        for i in range(0,len(data)):
            if data[i][pregunta_encuesta] == elemento:
                if data[i]["Sueldo en dolares"] != "":
                    sueldos.append(data[i]["Sueldo en dolares"])
                    sueldos = sorted(sueldos)
                if len(sueldos) ==1 or len(sueldos) == 0:
                    dic_sueldo_criterio[elemento] = sum(sueldos)
                elif len(sueldos)%2 != 0:
                    dic_sueldo_criterio[elemento] = (sueldos[(int(len(sueldos)/2))])
                elif len(sueldos)%2 == 0:
                    mediana = round((sueldos[int(len(sueldos)/2)]+sueldos[int(len(sueldos)/2-1)])/2,2)           
                    dic_sueldo_criterio[elemento] = (mediana)

   
    sorted_sueldo = sorted(dic_sueldo_criterio.values(), reverse =True)
    sorted_dic_sueldo_criterio = {}

    for participacion in sorted_sueldo:
        for elemento in dic_sueldo_criterio.keys():
            if dic_sueldo_criterio[elemento] == participacion:
                sorted_dic_sueldo_criterio[elemento] = dic_sueldo_criterio[elemento]

    
    return sorted_dic_sueldo_criterio

#Traigo la mediana de los sueldos para cada criterio usando la funcion
mediana_sueldo_por_rol = calculadora_mediana_sueldo(criterio = roles, pregunta_encuesta = "Trabajo de")
mediana_sueldo_por_genero = calculadora_mediana_sueldo(criterio = generos_depurado, pregunta_encuesta = "Me identifico")
mediana_sueldo_por_nivel_educativo = calculadora_mediana_sueldo(criterio = nivel_educativo, pregunta_encuesta ="Nivel de estudios alcanzado")


#SUELDOS TECNOLOGIAS POPULARES--------------------------------------------------------------------------------------------------
#Lenguajes de programacion
#formateo datos para unificar respuestas
for i in range(0, len(data)):
    data[i]["Lenguajes de programación o tecnologías."] = data[i]["Lenguajes de programación o tecnologías."].capitalize()
    if len(data[i]["Lenguajes de programación o tecnologías."]) == 1:
        data[i]["Lenguajes de programación o tecnologías."] = re.sub("[-.,0-9]", "", data[i]["Lenguajes de programación o tecnologías."])

lenguajes = respuestas_unicas(pregunta_encuesta = "Lenguajes de programación o tecnologías.")

#Identifico los lenguajes individuales dentro de cada respuesta
lenguajes  = list(lenguajes)
lenguajes_unicos = []
for i in range(0, len(lenguajes)):
    lenguajes_unicos.append(lenguajes[i].split(", "))
    
flattened = []
for sublist in lenguajes_unicos:
    for val in sublist:
        flattened.append(val)

#Limpio respuestas que no aplican
for i in range(0, len(flattened)):
    flattened[i] = flattened[i].capitalize()
    if flattened[i][-1] == ' ':
        flattened[i] = flattened[i][:-1]
    elif 'no' in flattened[i].lower():    
        flattened[i] = "No usa"
    elif "ninguno" in flattened[i].lower() or "ninguna" in flattened[i].lower():
        flattened[i] = "No usa"
        

lenguajes = set(flattened)

#Creo variacion de la calculadora de sueldo para los casos que tiene que buscar una palabra en el string en lugar de matchearla
def calculadora_mediana_sueldo_flex(criterio, pregunta_encuesta):
    sueldo_criterio = []
    for i in range(0,len(criterio)):
        sueldo_criterio.append(0)

    criterio = list(criterio)
    dic_sueldo_criterio = dict(zip(criterio, sueldo_criterio))
    #print(dic_sueldo_criterio)
    for elemento in criterio:
        sueldos = []
        for i in range(0,len(data)):
            if elemento in data[i][pregunta_encuesta]:
                if data[i]["Sueldo en dolares"] != "":
                    sueldos.append(data[i]["Sueldo en dolares"])
                    sueldos = sorted(sueldos)
                if len(sueldos) ==1 or len(sueldos) == 0:
                    dic_sueldo_criterio[elemento] = sum(sueldos)
                elif len(sueldos)%2 != 0:
                    dic_sueldo_criterio[elemento] = (sueldos[(int(len(sueldos)/2))])
                elif len(sueldos)%2 == 0:
                    mediana = round((sueldos[int(len(sueldos)/2)]+sueldos[int(len(sueldos)/2-1)])/2,2)           
                    dic_sueldo_criterio[elemento] = (mediana)

   
    sorted_sueldo = sorted(dic_sueldo_criterio.values(), reverse =True)
    sorted_dic_sueldo_criterio = {}

    for participacion in sorted_sueldo:
        for elemento in dic_sueldo_criterio.keys():
            if dic_sueldo_criterio[elemento] == participacion:
                sorted_dic_sueldo_criterio[elemento] = dic_sueldo_criterio[elemento]

    
    return sorted_dic_sueldo_criterio

#Sueldo Lenguajes
sorted_dic_sueldo_lenguajes = calculadora_mediana_sueldo_flex(criterio = lenguajes, pregunta_encuesta = "Lenguajes de programación o tecnologías.")

#Sueldo Plataformas
plataformas = respuestas_unicas(pregunta_encuesta = "Plataformas")

plataformas  = list(plataformas)
plataformas_unicos = []
for i in range(0, len(plataformas)):
    plataformas_unicos.append(plataformas[i].split(", "))
    
flattened = []
for sublist in plataformas_unicos:
    for val in sublist:
        if len(val) != 0: 
            if val.lower() != "ninguno" and val.lower() != "ninguna" and val.lower() != "no"\
            and val.lower() != "na" and val.lower() != "a" and val.lower() != "ninguna de las anteriores"\
            and 'eczel' not in val.lower():
                flattened.append(val)

plataformas = set(flattened)
sorted_dic_sueldo_plataformas = calculadora_mediana_sueldo_flex(criterio = plataformas, pregunta_encuesta = "Plataformas")

#Sueldo Frameworks
frameworks = respuestas_unicas(pregunta_encuesta = "Frameworks, herramientas y librerías")

frameworks  = list(frameworks)
frameworks_unicos = []
for i in range(0, len(frameworks)):
    frameworks_unicos.append(frameworks[i].split(", "))
    
flattened = []
for sublist in frameworks_unicos:
    for val in sublist:
        if len(val) != 0 and val != ".":
            if val[-1] == " ":
                val = val[:-1]
            if len(val) != 0 and val != ".": 
                if val.lower() != "ninguno" and val.lower() != "ninguna" and val.lower() != "no"\
                and val.lower() != "na" and val.lower() != "a" and val.lower() != "ninguno de los anteriores":
                    flattened.append(val)

frameworks = set(flattened)
sorted_dic_sueldo_frameworks = calculadora_mediana_sueldo_flex(criterio = frameworks, pregunta_encuesta = "Frameworks, herramientas y librerías")

#Sueldo Bases de datos
bases_de_datos = respuestas_unicas(pregunta_encuesta = "Bases de datos")

bases_de_datos  = list(bases_de_datos)
bases_de_datos_unicos = []
for i in range(0, len(bases_de_datos)):
    bases_de_datos_unicos.append(bases_de_datos[i].split(", "))
    
flattened = []
for sublist in bases_de_datos_unicos:
    for val in sublist:
        if len(val) != 0 and val != "etc)": 
            if val.lower() != "ninguno" and val.lower() != "ninguna" and val.lower() != "no"\
            and val.lower() != "na" and val.lower() != "a" and val.lower() != "ninguna de las anteriores":
                flattened.append(val)

bases_de_datos = set(flattened)
sorted_dic_sueldo_bases_de_datos = calculadora_mediana_sueldo_flex(criterio = bases_de_datos, pregunta_encuesta = "Bases de datos")

#Sueldo QA/ Testing
qa = respuestas_unicas(pregunta_encuesta = "QA / Testing")

qa  = list(qa)
qa_unicos = []
for i in range(0, len(qa)):
    qa_unicos.append(qa[i].split(", "))
    
flattened = []
for sublist in qa_unicos:
    for val in sublist:
        if len(val) != 0 and val != 'm': 
            if val.lower() != "ninguno" and val.lower() != "ninguna" and val.lower() != "no"\
            and val.lower() != "na" and val.lower() != "a" and val.lower() != "ninguna de las anteriores":
                flattened.append(val)

qa = set(flattened)
sorted_dic_sueldo_qa = calculadora_mediana_sueldo_flex(criterio = qa, pregunta_encuesta = "QA / Testing")

#Sueldo IDEs 
ide = respuestas_unicas(pregunta_encuesta = "IDEs")

ide  = list(ide)
ide_unicos = []
for i in range(0, len(ide)):
    ide_unicos.append(ide[i].split(", "))
    
flattened = []
for sublist in qa_unicos:
    for val in sublist:
        if 'otro' in val.lower():
            val = 'Otros'
        if len(val) != 0 and val.lower() != 'm' and val != '.' and val != '-' and val != '--'\
        and val != '0' and val != '123' and "no" not in val.lower() and 'ninguna' not in val.lower()\
        and val.lower() != "ninguno" and val.lower() != 'n/a' \
        and 'nada' not in val.lower() and val.lower() != "na" and val.lower() != "a" and\
        val.lower() != 'soy scrum master' and \
        'empresa' not in val.lower() and 'trabajo' not in val.lower():
            flattened.append(val)

ide = set(flattened)
sorted_dic_sueldo_ide = calculadora_mediana_sueldo_flex(criterio = ide, pregunta_encuesta = "IDEs")

#PARTICIPACION DE TECNOLOGIAS POPULARES--------------------------------------------------------------------------
#Defino funcion diccionarios buscando la palabra en lugar de matchear valor
def dic_respuestas_flex(criterio, pregunta_encuesta):
    participantes = []
    for i in range(0,len(criterio)):
        participantes.append(0)
    
    criterio = list(criterio)
    dic_respuestas = dict(zip(criterio, participantes))
    
    for i in range(0,len(data)):
        for elemento in criterio:
            if elemento in data[i][pregunta_encuesta]:
                dic_respuestas[elemento] += 1

    return dic_respuestas

dic_lenguajes = dic_respuestas_flex(criterio = lenguajes, pregunta_encuesta = "Lenguajes de programación o tecnologías.")
participacion_por_lenguajes = participacion_ordenada(dic_criterio = dic_lenguajes, criterio = lenguajes)

dic_plataformas = dic_respuestas_flex(criterio = plataformas, pregunta_encuesta = "Plataformas")
participacion_por_plataforma = participacion_ordenada(dic_criterio = dic_plataformas, criterio = plataformas)

dic_frameworks = dic_respuestas_flex(criterio = frameworks, pregunta_encuesta = "Frameworks, herramientas y librerías")
participacion_por_frameworks = participacion_ordenada(dic_criterio = dic_frameworks, criterio = frameworks)

dic_bases = dic_respuestas_flex(criterio = bases_de_datos, pregunta_encuesta = "Bases de datos")
participacion_por_bases = participacion_ordenada(dic_criterio = dic_bases, criterio = bases_de_datos)

dic_qa = dic_respuestas_flex(criterio = qa, pregunta_encuesta = "QA / Testing")
participacion_por_qa = participacion_ordenada(dic_criterio = dic_qa, criterio = qa)

dic_ide = dic_respuestas_flex(criterio = ide, pregunta_encuesta = "IDEs")
participacion_por_ide = participacion_ordenada(dic_criterio = dic_ide, criterio = ide)

#GRAFICAS-------------------------------------------------------------------------------------------------
#Defino funcion para graficar las participaciones mas populares y el salario medio correspondiente
def grafica(dic_participacion, dic_sueldo):
    largos_string = []
    for elemento in dic_participacion.keys():
        if round(dic_participacion[elemento])>1:
            largos_string.append(len(elemento))
    mas_largo = max(largos_string) + 1

    for elemento in dic_participacion.keys():
        if round(dic_participacion[elemento])>1:
            print(elemento, end="")
            for i in range(0, mas_largo - len(elemento)):
                print(" ", end="")
            for i in range(0, round(dic_participacion[elemento])):
                print('-', end="")
            print(f'{dic_participacion[elemento]}%  - Sueldo Medio: ${dic_sueldo[elemento]}')
            print()
         


In [245]:
#ENTREGA FINAL - RESPUESTAS
print(f'La encuesta sobre la que se basan las siguientes respuestas tiene una totalidad de {len(data)} respuestas')
#Analisis Rol vs Sueldo
print('Rol vs. Sueldo \n')
print('Grafica 1: Se presentan los roles con particpacion de al menos 1% en quienes respondieron la pregunta y \nsu Salario Medio en USD \n')
grafica(dic_participacion = participacion_por_rol, dic_sueldo = mediana_sueldo_por_rol)
print(f'El rol con mayor peso dentro de los encuestados que respondieron a la pregunta es {max(participacion_por_rol, key=participacion_por_rol.get)}, \n\
siendo el {max(participacion_por_rol.values())}% de los mismos. El salario medio del rol es de ${mediana_sueldo_por_rol[max(participacion_por_rol, key=participacion_por_rol.get)]}. \n\
Mientras que el rol con el salario medio mas alto es {max(mediana_sueldo_por_rol, key=mediana_sueldo_por_rol.get)} ganando en promedio ${mediana_sueldo_por_rol[max(mediana_sueldo_por_rol, key=mediana_sueldo_por_rol.get)]} y \
tiendo una participacion del {participacion_por_rol[max(mediana_sueldo_por_rol, key=mediana_sueldo_por_rol.get)]}% \
en los encuestados.\n\n')

#Nivel de educacion vs sueldo
print('Nivel de educacion alcanzado vs. Sueldo \n')
print('Grafica 2: Se presentan los niveles educativos con particpacion de al menos 1% en quienes respondieron la pregunta y \nsu Salario Medio en USD \n')
grafica(dic_participacion = participacion_por_nivel, dic_sueldo = mediana_sueldo_por_nivel_educativo)
print(f'El nivel educativo alcanzado con mayor peso dentro de los encuestados que respondieron a la pregunta es {max(participacion_por_nivel, key=participacion_por_nivel.get)}, \n\
siendo el {max(participacion_por_nivel.values())}% de los mismos. El salario medio de dicho nivel es de ${mediana_sueldo_por_nivel_educativo[max(participacion_por_nivel, key=participacion_por_nivel.get)]}. \n\
Mientras que el nivel educativo con el salario medio mas alto es {max(mediana_sueldo_por_nivel_educativo, key=mediana_sueldo_por_nivel_educativo.get)} ganando en promedio ${mediana_sueldo_por_nivel_educativo[max(mediana_sueldo_por_nivel_educativo, key=mediana_sueldo_por_nivel_educativo.get)]} \n\
y tiendo una participacion del {participacion_por_nivel[max(mediana_sueldo_por_nivel_educativo, key=mediana_sueldo_por_nivel_educativo.get)]}% en los encuestados.\n\
Este ultimo dato con respecto al salario llama mucho la atencion siendo que va encontra de lo que uno esperaria.\n\n')

#Genero vs sueldo
print('Genero vs. Sueldo \n')
print('Grafica 3: Se presentan generos con particpacion de al menos 1% en quienes respondieron la pregunta y \nsu Salario Medio en USD \n')      
grafica(dic_participacion = dic_participacion_genero, dic_sueldo = mediana_sueldo_por_genero)
print(f'El genero con mayor peso dentro de los encuestados que respondieron a la pregunta es {max(dic_participacion_genero, key=dic_participacion_genero.get)}, \n\
siendo el {max(dic_participacion_genero.values())}% de los mismos. El salario medio del mismo es de ${mediana_sueldo_por_genero[max(dic_participacion_genero, key=dic_participacion_genero.get)]}. \n\
Mientras que el genero con el salario medio mas alto es {max(mediana_sueldo_por_genero, key=mediana_sueldo_por_genero.get)} ganando en promedio ${mediana_sueldo_por_genero[max(mediana_sueldo_por_genero, key=mediana_sueldo_por_genero.get)]} \n\
y tiendo una participacion del {dic_participacion_genero[max(mediana_sueldo_por_genero, key=mediana_sueldo_por_genero.get)]}% en los encuestados.\n\n')


#Tecnologias populares
print('Tecnologias Populares: Lenguajes vs. Sueldo \n')
print('Grafica 4: Se presentan los lenguajes con particpacion de al menos 1% en quienes respondieron la pregunta y \nSalario Medio en USD de sus usuarios \n')
grafica(dic_participacion = participacion_por_lenguajes, dic_sueldo = sorted_dic_sueldo_lenguajes)
print(f'El lenguaje con mayor peso dentro de los encuestados que respondieron a la pregunta es {max(participacion_por_lenguajes, key=participacion_por_lenguajes.get)}, \n\
siendo el {max(participacion_por_lenguajes.values())}% de los mismos. El salario medio de los usuarios del mismo es de ${sorted_dic_sueldo_lenguajes[max(participacion_por_lenguajes, key=participacion_por_lenguajes.get)]}. \n\
Mientras que el lenguaje con los usuarios con salario promedio mas alto es {max(sorted_dic_sueldo_lenguajes, key=sorted_dic_sueldo_lenguajes.get)} \n\
ganando en promedio ${sorted_dic_sueldo_lenguajes[max(sorted_dic_sueldo_lenguajes, key=sorted_dic_sueldo_lenguajes.get)]} y tiendo una participacion del {participacion_por_lenguajes[max(sorted_dic_sueldo_lenguajes, key=sorted_dic_sueldo_lenguajes.get)]}% en los encuestados.\n\n')

print('Tecnologias Populares: Plataformas vs. Sueldo \n')
print('Grafica 5: Se presentan las plataformas con particpacion de al menos 1% en quienes utilizan alguna y \nSalario Medio en USD de sus usuarios \n')
grafica(dic_participacion = participacion_por_plataforma, dic_sueldo = sorted_dic_sueldo_plataformas)
print(f'La plataforma con mayor peso dentro de los encuestados que respondieron a la pregunta es {max(participacion_por_plataforma, key=participacion_por_plataforma.get)}, \n\
siendo el {max(participacion_por_plataforma.values())}% de los mismos. El salario medio de los usuarios de la mismo es de ${sorted_dic_sueldo_plataformas[max(participacion_por_plataforma, key=participacion_por_plataforma.get)]}. \n\
Mientras que la plataforma con los usuarios con salario promedio mas alto es {max(sorted_dic_sueldo_plataformas, key=sorted_dic_sueldo_plataformas.get)} \n\
ganando en promedio ${sorted_dic_sueldo_plataformas[max(sorted_dic_sueldo_plataformas, key=sorted_dic_sueldo_plataformas.get)]} y tiendo una participacion del \
{participacion_por_plataforma[max(sorted_dic_sueldo_plataformas, key=sorted_dic_sueldo_plataformas.get)]}% en los encuestados.\n\n')

print('Tecnologias Populares: Frameworks vs. Sueldo \n')
print('Grafica 6: Se presentan los frameworks con particpacion de al menos 1% en quienes utilizan alguno y \nSalario Medio en USD de sus usuarios\n')
grafica(dic_participacion = participacion_por_frameworks, dic_sueldo = sorted_dic_sueldo_frameworks)
print(f'El framework con mayor peso dentro de los encuestados que respondieron a la pregunta es {max(participacion_por_frameworks, key=participacion_por_frameworks.get)}, \n\
siendo el {max(participacion_por_frameworks.values())}% de los mismos. El salario medio de los usuarios del mismo es de ${sorted_dic_sueldo_frameworks[max(participacion_por_frameworks, key=participacion_por_frameworks.get)]}. \n\
Mientras que el framework con los usuarios con salario promedio mas alto es {max(sorted_dic_sueldo_frameworks, key=sorted_dic_sueldo_frameworks.get)} \n\
ganando en promedio ${sorted_dic_sueldo_frameworks[max(sorted_dic_sueldo_frameworks, key=sorted_dic_sueldo_frameworks.get)]} y tiendo una participacion del \
{participacion_por_frameworks[max(sorted_dic_sueldo_frameworks, key=sorted_dic_sueldo_frameworks.get)]}% en los encuestados.\n\n')


print('Tecnologias Populares: Bases de datos vs. Sueldo \n')
print('Grafica 7: Se presentan las bases de datos con particpacion de al menos 1% en quienes utilizan alguna y \nSalario Medio en USD de sus usuarios \n')
grafica(dic_participacion = participacion_por_bases, dic_sueldo = sorted_dic_sueldo_bases_de_datos)
print(f'Las bases de datos con mayor peso dentro de los encuestados que respondieron a la pregunta es {max(participacion_por_bases, key=participacion_por_bases.get)}, \n\
siendo el {max(participacion_por_bases.values())}% de los mismos. El salario medio de los usuarios de las mismas es de ${sorted_dic_sueldo_bases_de_datos[max(participacion_por_bases, key=participacion_por_bases.get)]}. \n\
Mientras que la base de datos con los usuarios con salario promedio mas alto es {max(sorted_dic_sueldo_bases_de_datos, key=sorted_dic_sueldo_bases_de_datos.get)} \n\
ganando en promedio ${sorted_dic_sueldo_bases_de_datos[max(sorted_dic_sueldo_bases_de_datos, key=sorted_dic_sueldo_bases_de_datos.get)]} y tiendo una participacion del \
{participacion_por_bases[max(sorted_dic_sueldo_bases_de_datos, key=sorted_dic_sueldo_bases_de_datos.get)]}% en los encuestados.\n\n')

print('Tecnologias Populares: Tecnologias de QA/ Testing vs. Sueldo \n')
print('Grafica 8: Se presentan las tecnologias de QA/Testing con particpacion de al menos 1% en quienes utilizan alguna y \nSalario Medio en USD de sus usuarios \n')
grafica(dic_participacion = participacion_por_qa, dic_sueldo = sorted_dic_sueldo_qa)
print(f'Las tecnologias de Testing con mayor peso dentro de los encuestados que respondieron a la pregunta es {max(participacion_por_qa, key=participacion_por_qa.get)}, \n\
siendo el {max(participacion_por_qa.values())}% de los mismos. El salario medio de los usuarios de las mismas es de ${sorted_dic_sueldo_qa[max(participacion_por_qa, key=participacion_por_qa.get)]}. \n\
Mientras que la tecnologia con los usuarios con salario promedio mas alto es {max(sorted_dic_sueldo_qa, key=sorted_dic_sueldo_qa.get)} \n\
ganando en promedio ${sorted_dic_sueldo_qa[max(sorted_dic_sueldo_qa, key=sorted_dic_sueldo_qa.get)]} y tiendo una participacion del \
{participacion_por_qa[max(sorted_dic_sueldo_qa, key=sorted_dic_sueldo_qa.get)]}% en los encuestados.\n\n')

print('Tecnologias Populares: Tecnologias de QA/ Testing vs. Sueldo \n')
print('Grafica 9: Se presentan las tecnologias de IDEs con particpacion de al menos 1% en quienes utilizan alguna y \nSalario Medio en USD de sus usuarios \n')
grafica(dic_participacion = participacion_por_ide, dic_sueldo = sorted_dic_sueldo_ide)
print(f'Las tecnologias de IDEs con mayor peso dentro de los encuestados que respondieron a la pregunta es {max(participacion_por_ide, key=participacion_por_ide.get)}, \n\
siendo el {max(participacion_por_ide.values())}% de los mismos. El salario medio de los usuarios de las mismas es de ${sorted_dic_sueldo_ide[max(participacion_por_ide, key=participacion_por_ide.get)]}. \n\
Mientras que la tecnologia con los usuarios con salario promedio mas alto es {max(sorted_dic_sueldo_ide, key=sorted_dic_sueldo_ide.get)} \n\
ganando en promedio ${sorted_dic_sueldo_ide[max(sorted_dic_sueldo_ide, key=sorted_dic_sueldo_ide.get)]} y tiendo una participacion del \
{participacion_por_ide[max(sorted_dic_sueldo_ide, key=sorted_dic_sueldo_ide.get)]}% en los encuestados.\n\
*Por la considerable cantidad de respuestas que tuvo, asumi que "O" es efectivamente el nombre de una IDE.\n\n')


La encuesta sobre la que se basan las siguientes respuestas tiene una totalidad de 6869 respuestas
Rol vs. Sueldo 

Grafica 1: Se presentan los roles con particpacion de al menos 1% en quienes respondieron la pregunta y 
su Salario Medio en USD 

Developer                 -----------------------------------------40.69%  - Sueldo Medio: $1431.84

SysAdmin / DevOps / SRE   -----------11.09%  - Sueldo Medio: $1555.54

Technical Leader          ------6.16%  - Sueldo Medio: $2290.95

QA / Tester               ----4.29%  - Sueldo Medio: $1234.01

Manager / Director        ----4.28%  - Sueldo Medio: $3054.6

Project Manager           ---3.09%  - Sueldo Medio: $2036.58

HelpDesk                  ---2.62%  - Sueldo Medio: $715.92

BI Analyst / Data Analyst --2.45%  - Sueldo Medio: $1240.93

Architect                 --2.23%  - Sueldo Medio: $2290.95

Consultant                --1.92%  - Sueldo Medio: $1240.93

Data Engineer             --1.76%  - Sueldo Medio: $1770.08

Business Analyst        